In [11]:
from ExploreData import ExploreData
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import typing as t

import os

In [12]:
print(os.getcwd())
data_path = '../data/'

df_ts = pd.read_csv(data_path + 'data_timeseries.csv')

display(df_ts.sample(5).sort_index())
print('shape:', df_ts.shape)

c:\Users\BrandonBrown\repos\basketball\scripts


,GAME_DATE_EST,GAME_ID,TEAM_ID,days_since_last_game,season_short,percentage_regular_season_games,next_0_HOME,next_0_days_until_next_game,next_0_PTS_ALLOWED,next_0_FG_PCT_ALLOWED,...,avg_3_22_REB_pm_season,avg_3_22_AST_pm_season,avg_3_22_STL_pm_season,avg_3_22_BLK_pm_season,avg_3_22_TO_pm_season,avg_3_22_PTS_pm_season,avg_3_22_PLUS_MINUS_pm_season,avg_3_22_PF_pm_season,avg_3_22_played_in_first_game_of_back_to_back_season,avg_3_22_played_in_second_game_of_back_to_back_season
30144,2004-11-12,20400078,1610612748,1,2005,0.073171,1.0,1.0,91.250000,0.417000,...,0.174743,0.053841,0.037243,0.013639,0.025694,0.177698,-1.091166,0.133843,0.776687,0.701535
50896,2005-03-16,20400943,1610612738,2,2005,0.792683,0.0,2.0,100.777778,0.465349,...,0.184744,0.041381,0.018573,0.029401,0.040373,0.296833,0.246499,0.109418,0.852425,0.835232
79071,2006-03-06,20500886,1610612763,1,2006,0.731707,1.0,3.0,99.716667,0.453400,...,0.136798,0.125895,0.048296,0.009908,0.050021,0.292980,-0.187550,0.100425,0.854580,0.824312
458919,2018-12-14,21800424,1610612754,2,2019,0.353659,1.0,2.0,112.000000,0.446000,...,0.094274,0.186627,0.038385,0.007505,0.078255,0.427882,0.006467,0.066387,0.954432,0.946546
465189,2019-01-18,21800677,1610612757,2,2019,0.573171,0.0,3.0,115.488889,0.465022,...,0.104090,0.053988,0.032128,0.008904,0.060860,0.383679,-0.146179,0.101474,0.809171,0.893149


shape: (579662, 274)


In [15]:
## Impute missing values in labels with 0
index_cols = ['GAME_DATE_EST', 'GAME_ID', 'TEAM_ID', 'PLAYER_ID']
all_labels = [
    'next_0_MIN', 'next_1_MIN', 'next_2_MIN', 'next_3_MIN', 'next_4_MIN',
    'next_0_PTS', 'next_1_PTS', 'next_2_PTS', 'next_3_PTS', 'next_4_PTS',
    'next_0_FGM', 'next_1_FGM', 'next_2_FGM', 'next_3_FGM', 'next_4_FGM',
    'next_0_FGA', 'next_1_FGA', 'next_2_FGA', 'next_3_FGA', 'next_4_FGA',
    'next_0_FTM', 'next_1_FTM', 'next_2_FTM', 'next_3_FTM', 'next_4_FTM',
    'next_0_FTA', 'next_1_FTA', 'next_2_FTA', 'next_3_FTA', 'next_4_FTA',
    'next_0_REB', 'next_1_REB', 'next_2_REB', 'next_3_REB', 'next_4_REB',
    'next_0_AST', 'next_1_AST', 'next_2_AST', 'next_3_AST', 'next_4_AST',
    'next_0_STL', 'next_1_STL', 'next_2_STL', 'next_3_STL', 'next_4_STL',
    'next_0_BLK', 'next_1_BLK', 'next_2_BLK', 'next_3_BLK', 'next_4_BLK',
    'next_0_TO', 'next_1_TO', 'next_2_TO', 'next_3_TO', 'next_4_TO',
    ]
do_not_include = [
    'next_0_FG3M', 'next_1_FG3M', 'next_2_FG3M', 'next_3_FG3M', 'next_4_FG3M',
    'next_0_FG3A', 'next_1_FG3A', 'next_2_FG3A', 'next_3_FG3A', 'next_4_FG3A',
    'next_0_PLUS_MINUS', 'next_1_PLUS_MINUS', 'next_2_PLUS_MINUS', 'next_3_PLUS_MINUS', 'next_4_PLUS_MINUS',
    'next_0_PF', 'next_1_PF', 'next_2_PF', 'next_3_PF', 'next_4_PF'
]

created_game_lag_cols = ['lag_1_START_POSITION', 'lag_1_comment_code', 'lag_1_comment_reason']
created_window_lag_cols = [
    'lag_1_MIN', 'lag_1_FGM_pm', 'lag_1_FGA_pm', 'lag_1_FG3M_pm', 'lag_1_FG3A_pm', 'lag_1_FTM_pm',
    'lag_1_FTA_pm', 'lag_1_REB_pm', 'lag_1_AST_pm', 'lag_1_STL_pm', 'lag_1_BLK_pm', 'lag_1_TO_pm',
    'lag_1_PTS_pm', 'lag_1_PLUS_MINUS_pm', 'lag_1_PF_pm', 'lag_1_game_played']
created_window_avg_cols = [
    'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_10_19_MIN', 'avg_20_35_MIN', 'avg_36_85_MIN',
    'avg_2_3_FGM_pm', 'avg_4_9_FGM_pm', 'avg_10_19_FGM_pm', 'avg_20_35_FGM_pm', 'avg_36_85_FGM_pm',
    'avg_2_3_FGA_pm', 'avg_4_9_FGA_pm', 'avg_10_19_FGA_pm', 'avg_20_35_FGA_pm', 'avg_36_85_FGA_pm',
    'avg_2_3_FG3M_pm', 'avg_4_9_FG3M_pm', 'avg_10_19_FG3M_pm', 'avg_20_35_FG3M_pm', 'avg_36_85_FG3M_pm',
    'avg_2_3_FG3A_pm', 'avg_4_9_FG3A_pm', 'avg_10_19_FG3A_pm', 'avg_20_35_FG3A_pm', 'avg_36_85_FG3A_pm',
    'avg_2_3_FTM_pm', 'avg_4_9_FTM_pm', 'avg_10_19_FTM_pm', 'avg_20_35_FTM_pm', 'avg_36_85_FTM_pm',
    'avg_2_3_FTA_pm', 'avg_4_9_FTA_pm', 'avg_10_19_FTA_pm', 'avg_20_35_FTA_pm', 'avg_36_85_FTA_pm',
    'avg_2_3_REB_pm', 'avg_4_9_REB_pm', 'avg_10_19_REB_pm', 'avg_20_35_REB_pm', 'avg_36_85_REB_pm',
    'avg_2_3_AST_pm', 'avg_4_9_AST_pm', 'avg_10_19_AST_pm', 'avg_20_35_AST_pm', 'avg_36_85_AST_pm',
    'avg_2_3_STL_pm', 'avg_4_9_STL_pm', 'avg_10_19_STL_pm', 'avg_20_35_STL_pm', 'avg_36_85_STL_pm',
    'avg_2_3_BLK_pm', 'avg_4_9_BLK_pm', 'avg_10_19_BLK_pm', 'avg_20_35_BLK_pm', 'avg_36_85_BLK_pm',
    'avg_2_3_TO_pm', 'avg_4_9_TO_pm', 'avg_10_19_TO_pm', 'avg_20_35_TO_pm', 'avg_36_85_TO_pm',
    'avg_2_3_PTS_pm', 'avg_4_9_PTS_pm', 'avg_10_19_PTS_pm', 'avg_20_35_PTS_pm', 'avg_36_85_PTS_pm',
    'avg_2_3_PLUS_MINUS_pm', 'avg_4_9_PLUS_MINUS_pm', 'avg_10_19_PLUS_MINUS_pm', 'avg_20_35_PLUS_MINUS_pm', 'avg_36_85_PLUS_MINUS_pm',
    'avg_2_3_PF_pm', 'avg_4_9_PF_pm', 'avg_10_19_PF_pm', 'avg_20_35_PF_pm', 'avg_36_85_PF_pm',
    'avg_2_3_game_played', 'avg_4_9_game_played', 'avg_10_19_game_played', 'avg_20_35_game_played', 'avg_36_85_game_played'
    ]
created_back_to_back_window_cols = [
    'avg_1_20_played_in_first_game_of_back_to_back', 'avg_21_90_played_in_first_game_of_back_to_back',
    'avg_1_20_played_in_second_game_of_back_to_back', 'avg_21_90_played_in_second_game_of_back_to_back',
    ]
players_season_stats_cols = ['lag_1_MIN_season', 'lag_2_MIN_season',
    'lag_1_game_played_season', 'lag_2_game_played_season',
    'lag_1_FGM_pm_season', 'lag_2_FGM_pm_season', 'lag_1_FGA_pm_season',
    'lag_2_FGA_pm_season', 'lag_1_FG3M_pm_season', 'lag_2_FG3M_pm_season',
    'lag_1_FG3A_pm_season', 'lag_2_FG3A_pm_season', 'lag_1_FTM_pm_season',
    'lag_2_FTM_pm_season', 'lag_1_FTA_pm_season', 'lag_2_FTA_pm_season',
    'lag_1_REB_pm_season', 'lag_2_REB_pm_season', 'lag_1_AST_pm_season',
    'lag_2_AST_pm_season', 'lag_1_STL_pm_season', 'lag_2_STL_pm_season',
    'lag_1_BLK_pm_season', 'lag_2_BLK_pm_season', 'lag_1_TO_pm_season',
    'lag_2_TO_pm_season', 'lag_1_PTS_pm_season', 'lag_2_PTS_pm_season',
    'lag_1_PLUS_MINUS_pm_season', 'lag_2_PLUS_MINUS_pm_season',
    'lag_1_PF_pm_season', 'lag_2_PF_pm_season',
    'lag_1_played_in_first_game_of_back_to_back_season',
    'lag_2_played_in_first_game_of_back_to_back_season',
    'lag_1_played_in_second_game_of_back_to_back_season',
    'lag_2_played_in_second_game_of_back_to_back_season',
    'avg_3_22_MIN_season', 'avg_3_22_game_played_season',
    'avg_3_22_FGM_pm_season', 'avg_3_22_FGA_pm_season',
    'avg_3_22_FG3M_pm_season', 'avg_3_22_FG3A_pm_season',
    'avg_3_22_FTM_pm_season', 'avg_3_22_FTA_pm_season',
    'avg_3_22_REB_pm_season', 'avg_3_22_AST_pm_season',
    'avg_3_22_STL_pm_season', 'avg_3_22_BLK_pm_season',
    'avg_3_22_TO_pm_season', 'avg_3_22_PTS_pm_season',
    'avg_3_22_PLUS_MINUS_pm_season', 'avg_3_22_PF_pm_season',
    'avg_3_22_played_in_first_game_of_back_to_back_season',
    'avg_3_22_played_in_second_game_of_back_to_back_season'
    ]
matchup_forecast_cols = [
    'next_0_HOME', 'next_0_days_until_next_game', 'next_0_PTS_ALLOWED', 'next_0_FG_PCT_ALLOWED', 'next_0_FG3_PCT_ALLOWED', 'next_0_AST_ALLOWED', 'next_0_REB_ALLOWED',
    'next_1_HOME', 'next_1_days_until_next_game', 'next_1_PTS_ALLOWED', 'next_1_FG_PCT_ALLOWED', 'next_1_FG3_PCT_ALLOWED', 'next_1_AST_ALLOWED', 'next_1_REB_ALLOWED',
    'next_2_HOME', 'next_2_days_until_next_game', 'next_2_PTS_ALLOWED', 'next_2_FG_PCT_ALLOWED', 'next_2_FG3_PCT_ALLOWED', 'next_2_AST_ALLOWED', 'next_2_REB_ALLOWED',
    'next_3_HOME', 'next_3_days_until_next_game', 'next_3_PTS_ALLOWED', 'next_3_FG_PCT_ALLOWED', 'next_3_FG3_PCT_ALLOWED', 'next_3_AST_ALLOWED', 'next_3_REB_ALLOWED',
    'next_4_HOME', 'next_4_days_until_next_game', 'next_4_PTS_ALLOWED', 'next_4_FG_PCT_ALLOWED', 'next_4_FG3_PCT_ALLOWED', 'next_4_AST_ALLOWED', 'next_4_REB_ALLOWED',
]
other_cols = [
    'days_since_last_game', 'season_short', 'percentage_regular_season_games',
    ]
# print cols that are in the lists and not the df, and vice versa
lists_ = index_cols + all_labels + do_not_include + created_game_lag_cols \
    + created_window_lag_cols + created_window_avg_cols + created_back_to_back_window_cols \
    + players_season_stats_cols + matchup_forecast_cols + other_cols
add_cols = [col for col in df_ts.columns if col not in lists_]
print(f'add col to lists ({len(add_cols)}):')
print(add_cols)
remove_cols = [col for col in lists_ if col not in df_ts.columns]
print(f'remove col from lists ({len(remove_cols)}):')
print(remove_cols)


add col to lists (0):
[]
remove col from lists (0):
[]


In [ ]:
df_ts[all_labels] = df_ts[all_labels].fillna(0)
df_ts

In [4]:
cat_features_min = ['HOME', 'lag_1_START_POSITION', 'lag_1_comment_code', 'lag_1_comment_reason', ]
num_features_min = [
    'days_since_last_game', 'days_until_next_game', 'season_short', 'percentage_regular_season_games', 'lag_1_MIN', # 'lag_1_WIN',
    'lag_1_PLUS_MINUS_pm', 'lag_1_PF_pm', # 'avg_2_3_WIN', 'avg_4_9_WIN', 'avg_10_19_WIN', 'avg_20_35_WIN', 'avg_36_85_WIN',
    'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_10_19_MIN', 'avg_20_35_MIN', 'avg_36_85_MIN',
    'avg_2_3_PLUS_MINUS_pm', 'avg_4_9_PLUS_MINUS_pm', 'avg_10_19_PLUS_MINUS_pm', 'avg_20_35_PLUS_MINUS_pm', 'avg_36_85_PLUS_MINUS_pm',
    'avg_2_3_PF_pm', 'avg_4_9_PF_pm', 'avg_10_19_PF_pm', 'avg_20_35_PF_pm', 'avg_36_85_PF_pm',
    'avg_1_20_played_in_first_game_of_back_to_back', 'avg_21_90_played_in_first_game_of_back_to_back', 'avg_1_20_played_in_second_game_of_back_to_back',
    'avg_21_90_played_in_second_game_of_back_to_back', 'avg_2_3_game_played', 'avg_4_9_game_played', 'avg_10_19_game_played', 'avg_20_35_game_played',
    'avg_36_85_game_played', 'lag_1_MIN_season', 'lag_2_MIN_season', 'lag_1_game_played_season', 'lag_2_game_played_season', # 'lag_1_WIN_season', 
    'lag_1_PLUS_MINUS_pm_season', 'lag_2_PLUS_MINUS_pm_season', 'lag_1_PF_pm_season', 'lag_2_PF_pm_season', 'lag_1_played_in_first_game_of_back_to_back_season',
    'lag_2_played_in_first_game_of_back_to_back_season', 'lag_1_played_in_second_game_of_back_to_back_season',
    'lag_2_played_in_second_game_of_back_to_back_season', 'avg_3_22_MIN_season', 'avg_3_22_game_played_season', 'avg_3_22_PLUS_MINUS_pm_season',
    'avg_3_22_PF_pm_season', 'avg_3_22_played_in_first_game_of_back_to_back_season', 'avg_3_22_played_in_second_game_of_back_to_back_season']
features_min = cat_features_min + num_features_min

cat_features_min_encoding = cat_features_min.copy()
cat_features_min_encoding.remove('HOME')

In [5]:
## TODO
## - Select columns to keep
var_dict_ts_min = {'num_vars': num_features_min + ['MIN'],
    'cat_vars': cat_features_min,
}

## - Explore data
explore_ts_min = ExploreData(df_ts, var_dict=var_dict_ts_min)
explore_ts_min.summary()
# explore_ts_min.target_variable(target='MIN')

========= Summary Statistics =========
579662 rows, 273 columns

========= Column Level Statistics =========
GAME_DATE_EST -- object
Column not in cat_vars or num_vars
Unique values: 3109
Null count: 0

GAME_ID -- int64
Column not in cat_vars or num_vars
Unique values: 23290
Null count: 0

TEAM_ID -- int64
Column not in cat_vars or num_vars
Unique values: 30
Null count: 0

TEAM_PTS -- float64
Column not in cat_vars or num_vars
Unique values: 105
Null count: 0

TEAM_FG_PCT -- float64
Column not in cat_vars or num_vars
Unique values: 418
Null count: 0

TEAM_FG3_PCT -- float64
Column not in cat_vars or num_vars
Unique values: 439
Null count: 0

TEAM_AST -- float64
Column not in cat_vars or num_vars
Unique values: 45
Null count: 0

TEAM_REB -- float64
Column not in cat_vars or num_vars
Unique values: 55
Null count: 0

OPP_TEAM_ID -- int64
Column not in cat_vars or num_vars
Unique values: 30
Null count: 0

OPP_TEAM_PTS -- float64
Column not in cat_vars or num_vars
Unique values: 105
Null co

In [6]:
# explore_ts_min.target_variable(target='MIN')

In [7]:
labels = ['MIN', 'PTS'] # ['MIN'] # 
df_ts_min = df_ts[labels + cat_features_min + num_features_min].copy()
df_ts_min

,MIN,PTS,HOME,lag_1_START_POSITION,lag_1_comment_code,lag_1_comment_reason,days_since_last_game,days_until_next_game,season_short,percentage_regular_season_games,...,lag_1_played_in_first_game_of_back_to_back_season,lag_2_played_in_first_game_of_back_to_back_season,lag_1_played_in_second_game_of_back_to_back_season,lag_2_played_in_second_game_of_back_to_back_season,avg_3_22_MIN_season,avg_3_22_game_played_season,avg_3_22_PLUS_MINUS_pm_season,avg_3_22_PF_pm_season,avg_3_22_played_in_first_game_of_back_to_back_season,avg_3_22_played_in_second_game_of_back_to_back_season
0,33.433333,19.0,0,NaN,NaN,NaN,6,1,2004,0.012195,...,NaN,NaN,NaN,NaN,37.245850,0.966117,0.040783,0.061334,0.957685,0.940702
1,33.183333,19.0,0,NaN,NaN,NaN,6,1,2004,0.012195,...,NaN,NaN,NaN,NaN,30.495325,0.922027,-0.009005,0.068593,0.930651,0.924901
2,13.700000,0.0,0,NaN,NaN,NaN,6,1,2004,0.012195,...,NaN,NaN,NaN,NaN,28.490421,0.879643,-0.066144,0.086062,0.922883,0.879047
3,28.983333,12.0,0,NaN,NaN,NaN,6,1,2004,0.012195,...,NaN,NaN,NaN,NaN,29.604933,0.917349,0.070746,0.065687,0.917152,0.924146
4,28.666667,8.0,0,NaN,NaN,NaN,6,1,2004,0.012195,...,NaN,NaN,NaN,NaN,24.926027,0.952234,0.036786,0.128981,0.958327,0.951249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579657,15.750000,11.0,0,NaN,NaN,NaN,2,0,2023,0.402439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579658,13.883333,4.0,0,NaN,NaN,NaN,2,0,2023,0.402439,...,0.909091,0.888889,0.909091,1.000000,26.919653,0.898917,0.006825,0.079604,0.888890,0.893185
579659,3.183333,0.0,0,NaN,DNP,Coach's Decision,2,0,2023,0.402439,...,0.500000,0.571429,0.500000,0.500000,12.915897,0.586235,-0.756749,0.123031,0.833333,0.500000
579660,3.183333,0.0,0,NaN,DNP,Coach's Decision,2,0,2023,0.402439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# train data is season_short 2007-2018 and 2021, validation data is 2019 and 2022, test data is 2020 and 2023
train_mask = (df_ts_min['season_short'].between(2007, 2018)) | (df_ts_min['season_short'] == 2021)
val_mask = (df_ts_min['season_short'] == 2019) | (df_ts_min['season_short'] == 2022)
test_mask = (df_ts_min['season_short'] == 2020)| (df_ts_min['season_short'] == 2023)
train_data_min = df_ts_min.loc[train_mask]
val_data_min = df_ts_min.loc[val_mask]
test_data_min = df_ts_min.loc[test_mask]

## seperate each of train, val, and test into their own X and y
X_train_min = train_data_min.drop(labels, axis=1)
y_train_min = train_data_min[labels]
X_val_min = val_data_min.drop(labels, axis=1)
y_val_min = val_data_min[labels]
X_test_min = test_data_min.drop(labels, axis=1)
y_test_min = test_data_min[labels]

feautes_min = X_train_min.columns.tolist()

print('train features shape:', X_train_min.shape)
print('train labels shape:', y_train_min.shape)
print('val data shape:', val_data_min.shape)
print('test data shape:', test_data_min.shape)

train features shape: (390303, 53)
train labels shape: (390303, 2)
val data shape: (62284, 55)
test data shape: (39793, 55)


In [9]:
from feature_engine.imputation import CategoricalImputer, AddMissingIndicator, MeanMedianImputer
from feature_engine.encoding import RareLabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

num_features_min_missing = [col for col in num_features_min if train_data_min[col].isnull().sum() > 0]
cat_features_min_missing = [col for col in cat_features_min if train_data_min[col].isnull().sum() > 0]
print(f'num_features_min_missing ({len(num_features_min_missing)}): {num_features_min_missing}')
print(f'cat_features_min_missing ({len(cat_features_min_missing)}): {cat_features_min_missing}')

print('Imputing categorical variables with missing data...')
cat_imputer = CategoricalImputer(
    imputation_method="missing",
    variables=cat_features_min_missing,
)
cat_imputer.fit(X_train_min)
X_train_min = cat_imputer.transform(X_train_min)
X_val_min = cat_imputer.transform(X_val_min)
X_test_min = cat_imputer.transform(X_test_min)

print('Adding missing indicator for categorical variables...')
num_missing_indicator = AddMissingIndicator(variables=num_features_min_missing)
num_missing_indicator.fit(X_train_min)
X_train_min = num_missing_indicator.transform(X_train_min)
X_val_min = num_missing_indicator.transform(X_val_min)
X_test_min = num_missing_indicator.transform(X_test_min)

print('Imputing numerical variables with mean...')
meanimputer = MeanMedianImputer(
    imputation_method="mean",
    variables=num_features_min_missing,
)
meanimputer.fit(X_train_min)
X_train_min = meanimputer.transform(X_train_min)
X_val_min = meanimputer.transform(X_val_min)
X_test_min = meanimputer.transform(X_test_min)

print('Encoding rare labels in categorical variables...')
encoder = RareLabelEncoder(
    tol=0.01, n_categories=1, variables=cat_features_min, ignore_format=True
)
encoder.fit(X_train_min)
X_train_min = encoder.transform(X_train_min)
X_val_min = encoder.transform(X_val_min)
X_test_min = encoder.transform(X_test_min)

## NOTE: new categories are automatically ignored
print('One hot encoding categorical variables...')
ohe = OneHotEncoder(
    drop_last_binary=True,
    variables=cat_features_min, ignore_format=True
)
ohe.fit(X_train_min)
X_train_min = ohe.transform(X_train_min)
X_val_min = ohe.transform(X_val_min)
X_test_min = ohe.transform(X_test_min)

col_names = X_train_min.columns.tolist()

print('Scaling numerical variables...')
scaler = MinMaxScaler()  # do you need a scaler with randomforest and lightgbm? -- no
scaler.fit(X_train_min)
X_train_min = pd.DataFrame(scaler.transform(X_train_min), columns=col_names)
X_val_min = pd.DataFrame(scaler.transform(X_val_min), columns=col_names)
X_test_min = pd.DataFrame(scaler.transform(X_test_min), columns=col_names)

print('train features shape:', X_train_min.shape)
print('train labels shape:', y_train_min.shape)
display(X_train_min.head())

num_features_min_missing (45): ['lag_1_MIN', 'lag_1_PLUS_MINUS_pm', 'lag_1_PF_pm', 'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_10_19_MIN', 'avg_20_35_MIN', 'avg_36_85_MIN', 'avg_2_3_PLUS_MINUS_pm', 'avg_4_9_PLUS_MINUS_pm', 'avg_10_19_PLUS_MINUS_pm', 'avg_20_35_PLUS_MINUS_pm', 'avg_36_85_PLUS_MINUS_pm', 'avg_2_3_PF_pm', 'avg_4_9_PF_pm', 'avg_10_19_PF_pm', 'avg_20_35_PF_pm', 'avg_36_85_PF_pm', 'avg_1_20_played_in_first_game_of_back_to_back', 'avg_21_90_played_in_first_game_of_back_to_back', 'avg_1_20_played_in_second_game_of_back_to_back', 'avg_21_90_played_in_second_game_of_back_to_back', 'avg_2_3_game_played', 'avg_4_9_game_played', 'avg_10_19_game_played', 'avg_20_35_game_played', 'avg_36_85_game_played', 'lag_1_MIN_season', 'lag_2_MIN_season', 'lag_1_game_played_season', 'lag_2_game_played_season', 'lag_1_PLUS_MINUS_pm_season', 'lag_2_PLUS_MINUS_pm_season', 'lag_1_PF_pm_season', 'lag_2_PF_pm_season', 'lag_1_played_in_first_game_of_back_to_back_season', 'lag_2_played_in_first_game_of_back_to_b

,days_since_last_game,days_until_next_game,season_short,percentage_regular_season_games,lag_1_MIN,lag_1_PLUS_MINUS_pm,lag_1_PF_pm,avg_2_3_MIN,avg_4_9_MIN,avg_10_19_MIN,...,lag_1_START_POSITION_F,lag_1_START_POSITION_C,lag_1_START_POSITION_G,lag_1_comment_code_Missing,lag_1_comment_code_DNP,lag_1_comment_code_DND,lag_1_comment_code_Rare,lag_1_comment_reason_Missing,lag_1_comment_reason_Coach's Decision,lag_1_comment_reason_Rare
0,0.25,0.0,0.0,0.0,0.38596,0.499777,0.001666,0.388032,0.329550,0.507020,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.25,0.0,0.0,0.0,0.38596,0.499777,0.001666,0.388032,0.329550,0.449927,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.25,0.0,0.0,0.0,0.38596,0.499777,0.001666,0.388032,0.432306,0.504274,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.25,0.0,0.0,0.0,0.38596,0.499777,0.001666,0.388032,0.432306,0.417356,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.25,0.0,0.0,0.0,0.38596,0.499777,0.001666,0.388032,0.432306,0.362157,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
from math import sqrt

# define base model
model = LinearSVR()
# define the chained multioutput wrapper model
chain = RegressorChain(model)
chain.fit(X_val_min, y_val_min)
yhat = chain.predict(X_test_min)

mse = mean_squared_error(y_test_min, yhat)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

RMSE: 7.050565


c:\Users\BrandonBrown\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [13]:
import joblib
joblib.dump(chain, 'chain_min.joblib')

chain2 = joblib.load('chain_min.joblib')
yhat2 = chain2.predict(X_test_min)
mse2 = mean_squared_error(y_test_min, yhat2)
print('RMSE: %f' % rmse)

RMSE: 7.050565


In [21]:
y_sample1 =y_test_min.reset_index().iloc[[1, 39789]]
X_sample1 = X_test_min.iloc[[1, 39789]]
yhat_sample1 = chain2.predict(X_sample1)
yhat_sample1 = pd.DataFrame(yhat_sample1, columns=labels)
yhat_sample1 = yhat_sample1.set_index(y_sample1.index)
yhat_sample1


,MIN,PTS
1,15.105878,8.627597
39789,18.592129,8.821126


In [ ]:
## TODO:
## -- DONE -- Test RegressorChain and MultiOutputRegressor to make sure it works
## -- DONE -- Test pickle works with RegressorChain and MultiOutputRegressor
## -- DONE -- Fill missing values in labels with 0s first thing in the script that creates data
## -- DONE -- Get rid of win features
## -- DONE -- Rewrite get_all_lags() to accept both positive and negative lags
## -- DONE -- Get future values for projects (e.g. 5 games out of days between games, back-to-back, opponent, and labels)
## -- SKIP - Can't Ordinal Encode with changing Y -- Change OHE to ordinal encoding for categorical variables
## If we are going Boruta ---
    ## Update boruta script to write to csv after each label
    ## Create test boruta dataset that can be run faster to test writing to csv
    ## Run Boruta for each label (future games probably not needed)
## If we are not going Boruta ---
    ## Test each label using SelectFromModel
## Select features -- any feautre that gets a 1 or 2 at least once is selected
## Build model with selected features

In [ ]:
# example of evaluating chained multioutput regression with an SVM model
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
# define dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define base model
model = LinearSVR()
# define the chained multioutput wrapper model
wrapper = RegressorChain(model)
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 0.636 (0.337)


In [22]:
## TODO - use multi-output regressor - https://machinelearningmastery.com/multi-output-regression-models-with-python/

In [35]:
# from boruta import BorutaPy
# from sklearn.ensemble import RandomForestRegressor

# # instantiate random forest
# forest = RandomForestRegressor(n_jobs = -1, max_depth = 5)

# # fit boruta
# boruta_selector = BorutaPy(forest, random_state = 0, n_estimators = 10) # 'auto')
# boruta_selector.fit(np.array(X_train_min), np.array(y_train_min).reshape(-1))

# # store results
# boruta_ranking = boruta_selector.ranking_
# selected_features = np.array(col_names)[boruta_ranking <= 2]
# selected_features

array(['percentage_regular_season_games', 'lag_1_MIN', 'avg_4_9_WIN',
       'avg_2_3_MIN', 'avg_4_9_MIN', 'avg_20_35_MIN',
       'avg_2_3_game_played', 'lag_1_MIN_season',
       'lag_1_played_in_first_game_of_back_to_back_season',
       'lag_1_MIN_na', 'lag_1_PLUS_MINUS_pm_na', 'lag_1_PF_pm_na',
       'lag_1_START_POSITION_Missing', 'lag_1_START_POSITION_F'],
      dtype='<U56')

In [32]:
## 73.3 sec to boruta with 10 estimators on test data
## 406.5 sec to boruta with 10 estimators on train data

array([ 1, 32, 46,  1, 54,  1,  8,  5, 54, 33, 28, 27, 36,  1,  1,  1,  1,
       25, 29, 10,  8, 12, 34, 30,  3, 12, 14, 38, 40, 30, 36, 39,  1,  1,
        1,  7, 18, 20,  1,  1,  1,  1,  1,  1,  1,  4,  1, 23, 21, 17,  6,
        2,  1,  1, 19, 23, 75,  1,  1,  1, 75, 75, 75, 75, 75, 22, 75, 75,
       75, 75, 15, 75, 54, 75, 54, 15, 75, 75, 75, 75, 43, 75, 26, 75, 75,
       75, 75, 75, 75, 46, 46, 75, 75, 46, 54, 54, 46, 75, 41, 75, 54, 42,
       75, 46, 75, 54, 54, 75,  1,  1, 35, 54,  1, 12, 75, 75,  1,  1,  1])